# WatchTower
In this notebook we will pull entities from watch tower.

In [3]:
# install all the azure identity modules
%pip install azure.identity
# install all the azure keyvault modules
%pip install azure-keyvault-secrets
# install pandas modules
%pip install pandas
# install all the azure sql database modules
%pip install pyodbc

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from azure.identity import AzureCliCredential, ChainedTokenCredential, DefaultAzureCredential, ManagedIdentityCredential
from azure.keyvault.secrets import SecretClient
from azure.identity import ClientSecretCredential
import struct
import pyodbc
import pandas as pd 
import requests;

class DatabaseConnection():
    def __init__(self, configuration : dict):
        server = configuration.get('server')
        if (not str(server).endswith('.database.windows.net')) :
            server = server + '.database.windows.net';
        self.sever = server;
        self.database = configuration.get('database');
        credential = ChainedTokenCredential(ManagedIdentityCredential(), 
                                            AzureCliCredential(), 
                                            DefaultAzureCredential());
        databaseToken = credential.get_token('https://database.windows.net/');
        print(databaseToken);
        tokenb = bytes(databaseToken[0], "UTF-8");
        exptoken = b'';
        for i in tokenb:
            exptoken += bytes({i});
            exptoken += bytes(1);
        tokenstruct = struct.pack("=i", len(exptoken)) + exptoken;
        connString = "Driver={ODBC Driver 17 for SQL Server};SERVER="+self.sever+";DATABASE="+self.database+"";
        SQL_COPT_SS_ACCESS_TOKEN = 1256; 
        self.connection = pyodbc.connect(connString, attrs_before = {SQL_COPT_SS_ACCESS_TOKEN:tokenstruct});


    # write a database to database, mode = append or replace
    def insert(self, df : pd.DataFrame, table :str):
        """
        Execute a SQL write.
        @param df: Data Frame.        
        @param table: Table Name.
        """
        statement = '''
        INSERT INTO {table}
        (
            {columns}
        )
        VALUES({place_holders})
        '''
        # add protection on column name
        columns = df.columns.tolist();
        for i in range(len(columns)):
            columns[i] = '[' + columns[i] + ']';
        statement = statement.format(table = table, 
                                     columns = (',').join(columns), 
                                     place_holders = (',').join(['?'] * len(df.columns)));
        print(statement);
        # Create a Pandas dataframe from the results
        cursor = self.connection.cursor();
        cursor.fast_executemany = True;
        cursor.executemany(statement, df.values.tolist());
        cursor.commit();
        cursor.close();

    # execute sql command
    def execute(self, statement : str):
        """
        Execute a SQL command.
        @param query: query.        
        """
        # Create a Pandas dataframe from the results
        cursor = self.connection.cursor();
        cursor.execute(statement);
        cursor.commit();
        cursor.close();


    def query(self, query: str) -> pd.DataFrame:
        """
        Execute a SQL query.
        @param query: The query.
        @returns: A Pandas DataFrame.
        """
        # Create a Pandas dataframe from the results
        
        df = pd.read_sql(query, self.connection);

        # Print the dataframe
        print(df);
        return df

    def close(self):
        """
        Close the Database client.
        """
        self.connection.close();

class KeyVault():
    def __init__(self, configuration : dict):
        key_vault = configuration.get('keyvault');
        credential = ChainedTokenCredential(ManagedIdentityCredential(), 
                                            AzureCliCredential(), 
                                            DefaultAzureCredential());
        kv_url = "https://{key_vault}.vault.azure.net";
        kv_url = kv_url.format(key_vault = key_vault);
        self.client = SecretClient(vault_url=kv_url, credential=credential)

    # write a database to database, mode = append or replace
    def get(self, secret_name : str) -> str:
        secret_value = self.client.get_secret(secret_name);
        return secret_value.value;


class RestAPI():
    def __init__(self, configuration : dict):
        app_id = configuration.get('app_id');
        app_secret = configuration.get('app_secret');
        tenant_id = '72f988bf-86f1-41af-91ab-2d7cd011db47';
        credential = ClientSecretCredential(tenant_id, app_id, app_secret);
        self.token = credential.get_token("280b00f4-2c5a-44d8-bab9-6acbbaaa7c78/.default").token;
    
    def call_data_api(self, uri : str) -> dict:
        result = [];
        try:
            headers = {
                "Authorization": "Bearer " + self.token,
                "Content-Type": "application/json"  # Adjust content type as per your API
            }
            response = requests.get(uri, headers=headers);
            if response.status_code == 200:
                result = response.json();
            else:
                print("Error:", response.status_code)
        except requests.exceptions.RequestException as e:
            print("Error:", e);
        return result;

key_vault = KeyVault({'keyvault' : 'etlframeworkkeyvaults'});
app_id = key_vault.get('sec-CoiDpxeETLClientId');
app_secret = key_vault.get('sec-CoiDpxeETLClientKey');
api_client = RestAPI({'app_id' : app_id, 'app_secret': app_secret});
result = api_client.call_data_api("https://api.infra.global.azure.com/pantheon/api/v2/entities/AGI_ProjectColos")
projects = result['result'];

# Create an empty DataFrame

df = pd.DataFrame(columns=['projectOid', 'snapshotDateId', 'coPlusIProjectId', 'fmColoId', 'coloTrancheId']);
table = [];
for p in projects:
    for t in p['tranches'] :
        record = [];
        record.append(p['projectOid']);
        record.append(t['snapshotDateId']);
        record.append(t['coPlusIProjectId']);
        record.append(t['fmColoId']);
        record.append(t['coloTrancheId']);
        table.append(record);
df = pd.DataFrame(table, columns=['projectOid', 'snapshotDateId', 'coPlusIProjectId', 'fmColoId', 'coloTrancheId']);
databaseConn = DatabaseConnection({'server' : 'capacityreporting.database.windows.net', 'database' :'bireportingdev'});
databaseConn.execute('DELETE FROM [STG_WatchTower].[ProjectColoTranches]')
databaseConn.insert(df, table = '[STG_WatchTower].[ProjectColoTranches]');
databaseConn.close();
print(df);


AccessToken(token='eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkwxS2ZLRklfam5YYndXYzIyeFp4dzFzVUhIMCIsImtpZCI6IkwxS2ZLRklfam5YYndXYzIyeFp4dzFzVUhIMCJ9.eyJhdWQiOiJodHRwczovL2RhdGFiYXNlLndpbmRvd3MubmV0LyIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzcyZjk4OGJmLTg2ZjEtNDFhZi05MWFiLTJkN2NkMDExZGI0Ny8iLCJpYXQiOjE3MTUzNzAwOTEsIm5iZiI6MTcxNTM3MDA5MSwiZXhwIjoxNzE1Mzc0ODAxLCJfY2xhaW1fbmFtZXMiOnsiZ3JvdXBzIjoic3JjMSJ9LCJfY2xhaW1fc291cmNlcyI6eyJzcmMxIjp7ImVuZHBvaW50IjoiaHR0cHM6Ly9ncmFwaC53aW5kb3dzLm5ldC83MmY5ODhiZi04NmYxLTQxYWYtOTFhYi0yZDdjZDAxMWRiNDcvdXNlcnMvOTJlYzgxNWYtZGM0Yy00ZWZmLWFmNGYtMzRjMjc5ZGI4YmIzL2dldE1lbWJlck9iamVjdHMifX0sImFjciI6IjEiLCJhaW8iOiJBZVFBRy84V0FBQUFjcUlwL0tkbFZuLzRkY0E1aTdSMXMrQ1hsVG11eHdKdGxBZnAxcWFzcDRnYkJ5amUyZXdVSU1SRVNNdm1RbDRqdmU5eUVxdmNpMk5udWxmUTB4cEZZRGt1Q1EyYnRYTVdVL1p4NmJ6ajVJdjd5cWZHMlROakJEYTRBNmFpTjJBMXRmemhZMlo0cjV5a1p2M3lKdnM3b3lBbi9tRm1MaERoZlhhbDRhejVwT2dqWHdTTklIN0VFTHNoQXFHNXFTcGJsa0JEbW90QXgxbk01bkw4a01UUTVoekhPOTFEQlNYVkpGdzA4UzE3clY5YVpoTlRPNndGYTc1YTVk